```
pip3 install .
```

from within the Autorecommender folder.

Requirements:

-        "Keras >= 2.1.3",
-        "pandas >= 0.23.0",
-        "numpy >= 1.14.2",
-        "tensorflow >= 1.4.0",
-        "scipy >= 1.1.0"

In [1]:
import os
print(os.getcwd())

/Users/mahowald/ODG/bofa-poc-repo/models/notebooks


In [2]:
import pandas as pd
import numpy as np
import math

/Users/mahowald/anaconda2/envs/python3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
from autorecommender.models.autoencoder import load_model # need this to load the model

/Users/mahowald/anaconda2/envs/python3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/mahowald/anaconda2/envs/python3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/mahowald/anaconda2/envs/python3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/Users/mahowald/anaconda2/envs/python3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/mahowal

In [4]:
DATA_PATH = '../data/'
STOCKS_CSV = DATA_PATH + 'stocks.csv'
USERS_CSV = DATA_PATH + 'train_users.csv'
RATINGS_CSV = DATA_PATH + 'train_ratings.csv'

In [5]:
RATINGS = pd.read_csv(RATINGS_CSV)
RATINGS = RATINGS.set_index(['userid', 'stockid'])

STOCKS = pd.read_csv(STOCKS_CSV)
STOCKS = STOCKS.set_index('stockid').sort_index()

STOCKS_TICKER_LOOKUP = STOCKS.reset_index().set_index('ticker')

USERS = pd.read_csv(USERS_CSV).set_index('userid')

How to make a prediction with the model

In [6]:
def stockid_to_stockname(number):
    x = STOCKS.loc[number]
    return x['ticker']

In [7]:
# Avro input schema:
{
    "type": "record",
    "name": "ratings",
    "fields": [
        {"type": "string", "name": "userid"},
        {"type": {"type": "map", "items": "double"}} # probably - check map syntax
    ]
}

# Avro output schema:
{"type": "array", "items": "string"}

# fastscore input looks like:
# {'userid': 123, 'ratings': [{'AABA': 4}, {'AAL': 3}, {'AAON': 5}]}

# fastscore output looks like:
# ['AAA', 'BBB', 'CCC', 'DDD']


{'fields': [{'name': 'userid', 'type': 'string'},
  {'type': {'items': 'double', 'type': 'map'}}],
 'name': 'ratings',
 'type': 'record'}

In [8]:
# convert input to appropriate form for the model

def df_from_fastscore(x):
    userid = x['userid']
    ratings_list = []
    for r in x['ratings']:
        key = list(r.keys())[0]
        val = list(r.values())[0]
        ratings_list.append({'userid': userid, 
                             'stockid': STOCKS_TICKER_LOOKUP.loc[key]['stockid'].iloc[0], 
                             'rating' : val})
    return pd.DataFrame(ratings_list).set_index(['userid', 'stockid'])

This is a dataframe with a multiindex (userid, stockid) with a column named "rating" --> this is what the inputs FastScore receives need to be translated into.

In [9]:
trained_model = load_model('model.zip')

In [10]:
model_inputs = df_from_fastscore({'userid': 123,
                                 'ratings': [
                                     {'AABA': 4},
                                     {'AAL': 3},
                                     {'AAON': 5}
                                 ]})

In [11]:
predictions = trained_model.predict(model_inputs)
predictions

stockid,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
userid,,,,,,,,,,,,,,,,,,,,,
123,4.650565,3.759466,3.633219,3.335773,3.60591,4.391133,4.042272,3.673759,3.263567,4.058906,...,3.592536,2.908781,2.117535,2.926465,4.129988,4.172725,4.719203,4.210388,4.632974,4.36407


In [18]:
def predictions_to_recommendations(preds, userid):
    stockids = preds.transpose().sort_values(by=userid, ascending=False).index[0:10]
    outs = [stockid_to_stockname(x) for x in stockids]
    return outs
    

In [19]:
predictions_to_recommendations(predictions, 123)

['LACQU',
 'PME',
 'VCSH',
 'ELTK',
 'BRPAU',
 'VRNT',
 'CVLY',
 'UCBA',
 'XCRA',
 'CGO']

## FastScore Action function

In [20]:
def action(x):
    userid = x['userid']
    inputs = df_from_fastscore(x)
    yield predictions_to_recommendations(trained_model.predict(inputs), userid)

In [21]:
next(action({'userid': 123,
             'ratings': [
                 {'AABA': 4},
                 {'AAL': 3},
                 {'AAON': 5}
             ]}))

['LACQU',
 'PME',
 'VCSH',
 'ELTK',
 'BRPAU',
 'VRNT',
 'CVLY',
 'UCBA',
 'XCRA',
 'CGO']

## Fastscore Begin Function

In [22]:
def begin():
    global trained_model, STOCKS, STOCKS_TICKER_LOOKUP
    trained_model = load_model('model.zip')
    
    DATA_PATH = '../data/' # Update this if necessary
    STOCKS_CSV = DATA_PATH + 'stocks.csv'

    STOCKS = pd.read_csv(STOCKS_CSV)
    STOCKS = STOCKS.set_index('stockid').sort_index()

    STOCKS_TICKER_LOOKUP = STOCKS.reset_index().set_index('ticker')

Other functions to include in the FastScore model execution script:

* `stockid_to_stockname`
* `predictions_to_recommendations`
* `df_from_fastscore`

Files that need to be in the attachment:

* `model.zip`
* `stocks.csv`